<a href="https://colab.research.google.com/github/sean-halpin/chess_website/blob/models_init/models/Chess_Eval_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Data Download

In [ ]:
# https://database.lichess.org/standard/lichess_db_standard_rated_2013-01.pgn.zst

In [ ]:
!pip install zstandard

In [ ]:
import requests
import zstandard
import io

lichess = "https://database.lichess.org/standard/"
dbfile = "lichess_db_standard_rated_2017-01.pgn.zst"
url = lichess + dbfile
compressed_file_path = dbfile
output_path = dbfile.replace(".zst","")

# Download the Zstandard compressed file
response = requests.get(url)
if response.status_code == 200:
    with open(compressed_file_path, 'wb') as file:
        file.write(response.content)
    print(f"File downloaded successfully to {compressed_file_path}")
else:
    print(f"Failed to download the file. Status code: {response.status_code}")

# Extract the Zstandard compressed file
with open(compressed_file_path, 'rb') as compressed_file:
    dctx = zstandard.ZstdDecompressor()
    with dctx.stream_reader(compressed_file) as reader:
        with open(output_path, 'wb') as output:
            output.write(reader.read())

print(f"File extracted successfully to {output_path}")


# Data Exploration

In [ ]:
!ls

In [ ]:
!cat lichess_db_standard_rated_2017-01.pgn | grep '\[%eval' | wc -l

In [ ]:
!cat lichess_db_standard_rated_2017-01.pgn | grep '\[%eval' | head -n5

In [ ]:
!cat lichess_db_standard_rated_2017-01.pgn | grep '\[%eval' > lichess.pgn.eval

In [ ]:
!pip install python-chess

# Data Visualisation

In [ ]:
import chess.pgn
import re

pgn_text = """
1. e4 { [%eval 0.2] [%eval 0.2] } 1... e6 { [%eval 0.13] } 2. Bc4 { [%eval -0.31] } 2... d5 { [%eval -0.28] } 3. exd5 { [%eval -0.37] } 3... exd5 { [%eval -0.31] } 4. Bb3 { [%eval -0.33] } 4... Nf6 { [%eval -0.35] } 5. d4 { [%eval -0.34] } 5... Be7 { [%eval 0.0] } 6. Nf3 { [%eval 0.0] } 6... O-O { [%eval -0.08] } 7. Bg5 { [%eval -0.19] } 7... h6 { [%eval -0.29] } 8. Bxf6 { [%eval -0.36] } 8... Bxf6 { [%eval -0.37] } 9. O-O { [%eval -0.36] } 9... c6 { [%eval -0.12] } 10. Re1 { [%eval -0.17] } 10... Bf5 { [%eval -0.04] } 11. c4?! { [%eval -0.67] } 11... dxc4 { [%eval -0.5] } 12. Bxc4 { [%eval -0.77] } 12... Nd7?! { [%eval -0.1] } 13. Nc3 { [%eval 0.0] } 13... Nb6 { [%eval 0.0] } 14. b3?! { [%eval -0.76] } 14... Nxc4 { [%eval -0.49] } 15. bxc4 { [%eval -0.65] } 15... Qa5 { [%eval -0.55] } 16. Rc1 { [%eval -0.79] } 16... Rad8 { [%eval -0.78] } 17. d5?? { [%eval -5.41] } 17... Bxc3 { [%eval -5.42] } 18. Re5? { [%eval -7.61] } 18... Bxe5 { [%eval -7.78] } 19. Nxe5 { [%eval -7.72] } 19... cxd5 { [%eval -7.81] } 20. Qe1? { [%eval -9.29] } 20... Be6?? { [%eval 3.71] } 21. Rd1?? { [%eval -12.34] } 21... dxc4 { [%eval -12.71] } 22. Rxd8?! { [%eval #-1] } 22... Rxd8?! { [%eval -13.06] } 23. Qc3?! { [%eval #-2] } 23... Qxc3?! { [%eval #-4] } 24. g3 { [%eval #-3] } 24... Rd1+?! { [%eval #-4] } 25. Kg2 { [%eval #-4] } 25... Qe1?! { [%eval #-4] } 26. Kf3 { [%eval #-3] } 26... Qxe5 { [%eval #-2] } 27. Kg2 { [%eval #-2] } 27... Bd5+?! { [%eval #-2] } 28. Kh3 { [%eval #-1] } 28... Qh5# 0-1
"""

game = chess.pgn.read_game(io.StringIO(pgn_text))

board = game.board()
for node in list(game.mainline()):
    print()
    print(node.move)
    # print(node)
    print(node.comment)
    pattern = r"\[%eval\s+([#])*([\d.-]+)\]"
    match = re.search(pattern, node.comment)
    if match:
        print((match.group(1) if match.group(1) is not None else ""), match.group(2))
    else:
        print("No '%eval' found in the text.")
    board.push(node.move)
    print(board)
    checkMate = board.is_checkmate()
    print("Checkmate?", checkMate)
    i=0
    for square in chess.SQUARES:
      piece = board.piece_at(square)
      print(f"Square {chess.square_name(square)}: {piece}: {(board.piece_at(square).piece_type * (1 if board.piece_at(square).color else -1)) if piece is not None else 0}")


In [ ]:
abs(float('-2.0'))

In [ ]:
bound = 10

In [ ]:
def bound_value(value):
    return min(max(value, -bound), bound)

# Data Preparation

In [ ]:
import pandas as pd
rows_data = []
columns = []
for square in chess.SQUARES:
  piece = board.piece_at(square)
  columns.append(chess.square_name(square))
columns.append("eval")

i=0
with open('lichess.pgn.eval', 'r') as file:
    for pgn_text in file:
      # print(pgn_text)
      game = chess.pgn.read_game(io.StringIO(pgn_text))
      board = game.board()
      for node in list(game.mainline()):
          row_data = [(((board.piece_at(square).piece_type * (1 if board.piece_at(square).color else -1))) if board.piece_at(square) is not None else 0) for square in chess.SQUARES]
          pattern = r"\[%eval\s+([#])*([\d.-]+)\]"
          match = re.search(pattern, node.comment)
          if match:
            if match.group(1) is None:
              eval = bound_value(float(f"{match.group(2)}"))
              check_in = 0
            else:
              eval = 0
              if float(match.group(2)) > 0 and float(match.group(2)) <= bound:
                check_in = (bound + 10) - float(match.group(2))
              elif float(match.group(2)) < 0 and float(match.group(2)) >= -bound:
                check_in = (-bound + 10) + float(match.group(2))
              else:
                check_in = 0
              eval = check_in

          whitePlays = board.turn
          board.push(node.move)
          if board.is_checkmate():
            eval += (50 * (-1 if whitePlays else 1))
          row_data.append(eval)
          rows_data.append(row_data)
          # move next
      i+=1
      if i % 1_000 == 0:
        print(i)
      if i >= 15_000:
        break

df = pd.DataFrame(rows_data, columns=columns)

In [ ]:
df

In [ ]:
value_ranges = df.describe().loc[['min', 'max']]
print("Value Ranges:")
print(value_ranges)

In [ ]:
df.dtypes

# Define Model

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

class Net(nn.Module):
    def __init__(self, input_dim):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(input_dim, 1024)
        self.fc2 = nn.Linear(1024, 512)
        self.fc2z = nn.Linear(512, 512)
        self.fc2a = nn.Linear(512, 256)
        self.fc2b = nn.Linear(256, 128)
        self.fc3_eval = nn.Linear(128, 1)  # Output for 'eval'

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = torch.relu(self.fc2z(x))
        x = torch.relu(self.fc2a(x))
        x = torch.relu(self.fc2b(x))
        eval_output = self.fc3_eval(x)
        return eval_output


In [ ]:
import gc
gc.collect()
torch.cuda.empty_cache()

# Prepare Train & Test Data

In [ ]:
# Check if GPU is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

sampled_df = df.sample(n=1_000_000)

# Prepare the data
X = sampled_df.drop(columns=['eval']).values
y = sampled_df[['eval']].values

# Split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Normalize the data
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Convert data to PyTorch tensors and move to GPU if available
X_train_tensor = torch.tensor(X_train, dtype=torch.float32).to(device)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32).to(device)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32).to(device)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32).to(device)

# Define the model and move it to GPU if available
input_dim = X_train.shape[1]
model = Net(input_dim).to(device)

# Define loss function and optimizer
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)


# Train Model

In [ ]:
# Train the model
epochs = 200
for epoch in range(epochs):
    model.train()
    optimizer.zero_grad()
    eval_output = model(X_train_tensor)

    # Compute the loss for 'eval' output
    eval_loss = criterion(eval_output, y_train_tensor[:, 0].view(-1, 1))  # Assuming 'eval' is the first column
    loss = eval_loss

    loss.backward()
    optimizer.step()

    if epoch % 10 == 0:
        print(f'Epoch {epoch+1}/{epochs}, Loss: {loss.item()}')

# Evaluate the model
model.eval()
with torch.no_grad():
    eval_output = model(X_test_tensor)

    # Compute the loss for 'eval' output
    eval_loss = criterion(eval_output, y_test_tensor[:, 0].view(-1, 1))  # Assuming 'eval' is the first column
    test_loss = eval_loss

    print(f'Test Loss: {test_loss.item()}')

# Play Chess with Model

In [ ]:
def generate_pgn(board, moves):
    game = chess.pgn.Game()

    node = game
    for move in moves:
        node = node.add_variation(move)

    game.headers["Result"] = board.result()
    return str(game)

In [ ]:
import chess
import chess.svg
import torch
import torch.nn as nn
from IPython.display import SVG, display

# Check if GPU is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Define a function to convert the board state into a format suitable for input to the neural network
def preprocess_board_state(board):
    board_state = [(((board.piece_at(square).piece_type * (1 if board.piece_at(square).color else -1))) if board.piece_at(square) is not None else 0) for square in chess.SQUARES]
    return torch.tensor(board_state, dtype=torch.float32).to(device)

# Define a function to generate legal moves for the current player
def generate_legal_moves(board):
    legal_moves = []
    for move in board.legal_moves:
        legal_moves.append(move)
    return legal_moves

def minimax():
  return None

# Evaluate each legal move using the neural network model and select the move with the highest evaluation
def select_best_move(board, legal_moves, model):
    whitePlays = board.turn
    best_move = None
    best_eval = float('-inf') if board.turn else float('inf')  # Initialize best evaluation based on player's perspective

    for move in legal_moves:
        board.push(move)
        board_state = preprocess_board_state(board)
        eval_output = model(board_state)

        # Assuming 'eval_output' contains the evaluation for the current player's perspective
        eval_value = eval_output.item()

        if whitePlays:  # Maximizing player
            if eval_value > best_eval:
                best_eval = eval_value
                best_move = move
        else:  # Minimizing player
            if eval_value < best_eval:
                best_eval = eval_value
                best_move = move

        board.pop()
    print(f"EVAL: {best_eval}: {best_move}")
    return best_move
def minimax(board, legal_moves, model, depth, max_depth, alpha, beta, maximizing_player):
    if depth == max_depth or board.is_game_over():
        return select_best_move(board, legal_moves, model)

    if maximizing_player:
        max_eval = float('-inf')
        for move in legal_moves:
            board.push(move)
            eval_output = model(preprocess_board_state(board))
            eval_value = eval_output.item()
            max_eval = max(max_eval, minimax(board, generate_legal_moves(board), model, depth + 1, max_depth, alpha, beta, False))
            alpha = max(alpha, max_eval)
            board.pop()
            if beta <= alpha:
                break
        return max_eval
    else:
        min_eval = float('inf')
        for move in legal_moves:
            board.push(move)
            eval_output = model(preprocess_board_state(board))
            eval_value = eval_output.item()
            min_eval = min(min_eval, minimax(board, generate_legal_moves(board), model, depth + 1, max_depth, alpha, beta, True))
            beta = min(beta, min_eval)
            board.pop()
            if beta <= alpha:
                break
        return min_eval

def play_game(model, max_depth):
    board = chess.Board()
    moves = []
    while not board.is_game_over():
        if board.is_checkmate() or board.is_stalemate() or board.is_insufficient_material() or board.is_seventyfive_moves():
            print("Game Over")
            print("Result: ", board.result())
            return

        legal_moves = generate_legal_moves(board)
        if len(legal_moves) == 0:
            print("Game Over - No legal moves left")
            print("Result: ", board.result())
            return

        best_move = select_best_move(board, legal_moves, model)
        board.push(best_move)
        moves.append(best_move)
        print("**************************************", "\n")
        display(SVG(chess.svg.board(board,size=300)))

    # Print the final result of the game
    print("Game Over")
    print("Result: ", board.result())
    print(generate_pgn(board, moves))  # Generate and print the PGN

# Example usage:
# Assuming 'model' is your trained neural network model
play_game(model.to(device), max_depth=3)  # Move the model to GPU if available and set max depth to desired value

